# Azure AI ഏജന്റ് സേവനവും സെമാന്റിക് കർണലും ഉപയോഗിച്ച് RAG

ഈ കോഡ് സ്നിപ്പറ്റ്, `Azure AI Agent Service`യും `Semantic Kernel`ഉം ഉപയോഗിച്ച് Retrieval-Augmented Generation (RAG) നടത്തുന്നതിനുള്ള Azure AI ഏജന്റിനെ സൃഷ്ടിക്കുകയും നിയന്ത്രിക്കുകയും ചെയ്യുന്നതിന് എങ്ങനെ പ്രവർത്തിക്കാമെന്ന് കാണിക്കുന്നു. ഏജന്റ്, തിരഞ്ഞെടുത്ത കോൺടെക്സ്റ്റ് അടിസ്ഥാനമാക്കി ഉപയോക്തൃ ചോദ്യങ്ങൾ പ്രോസസ്സ് ചെയ്ത് അതിനനുസരിച്ച് കൃത്യമായ മറുപടികൾ നൽകുന്നു.


## പരിസ്ഥിതി ആരംഭിക്കുന്നു


SQLite പതിപ്പ് പരിഹാരം  
നിങ്ങൾക്ക് താഴെ കാണുന്ന പിശക് നേരിടുന്നുണ്ടെങ്കിൽ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
നിങ്ങളുടെ നോട്ട്‌ബുക്കിന്റെ തുടക്കത്തിൽ ഈ കോഡ് ബ്ലോക്ക് അൺകമ്മെന്റ് ചെയ്യുക:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുന്നു  
താഴെ കൊടുത്തിരിക്കുന്ന കോഡ് ആവശ്യമായ പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുന്നു:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# സെമാന്റിക് കർണൽ & Azure AI ഏജന്റ് സർവീസ് ഉപയോഗിച്ച് Retrieval-Augmented Generation

ഈ ഉദാഹരണം **Azure AI ഏജന്റ് സർവീസ്** ഉപയോഗിച്ച് **Retrieval-Augmented Generation (RAG)** എങ്ങനെ നടത്താമെന്ന് ഒരു ഭാഷാ മോഡലിനെ അപ്‌ലോഡ് ചെയ്ത ഡോക്യുമെന്റിൽ നിന്നുള്ള ഡൊമെയ്ൻ-സ്പെസിഫിക് കോൺടെക്സ്റ്റുമായി സംയോജിപ്പിച്ച് കാണിക്കുന്നു.

### ഇത് എങ്ങനെ പ്രവർത്തിക്കുന്നു

1. **ഡോക്യുമെന്റ് അപ്‌ലോഡ്**: ഒരു markdown ഫയൽ (document.md) വിവരങ്ങൾ (Contosoയുടെ യാത്രാ ഇൻഷുറൻസ് നയം) അടങ്ങിയതായും ഏജന്റ് സർവീസിലേക്ക് അപ്‌ലോഡ് ചെയ്യപ്പെടുന്നു.

2. **വെക്ടർ സ്റ്റോർ സൃഷ്ടി**: ഡോക്യുമെന്റ് അതിന്റെ ഉള്ളടക്കത്തിൽ സെമാന്റിക് തിരച്ചിൽ സാധ്യമാക്കാൻ വെക്ടർ സ്റ്റോറിലേക്ക് ഇൻഡക്സ് ചെയ്യപ്പെടുന്നു.

3. **ഏജന്റ് കോൺഫിഗറേഷൻ**: `gpt-4o` മോഡൽ ഉപയോഗിച്ച് ഒരു ഏജന്റ് താഴെ പറയുന്ന കർശനമായ നിർദ്ദേശങ്ങളോടെ സജ്ജമാക്കുന്നു:
   - ഡോക്യുമെന്റിൽ നിന്ന് തിരികെ ലഭിച്ച ഉള്ളടക്കത്തെ അടിസ്ഥാനമാക്കി മാത്രമേ ചോദ്യങ്ങൾക്ക് ഉത്തരം നൽകൂ.
   - ചോദ്യങ്ങൾ പരിധിക്ക് പുറത്താണെങ്കിൽ ഉത്തരം നൽകാൻ നിരസിക്കുക.

4. **ഫയൽ സെർച്ച് ടൂൾ ഇന്റഗ്രേഷൻ**: `FileSearchTool` ഏജന്റുമായി രജിസ്റ്റർ ചെയ്യപ്പെടുന്നു, മോഡലിന് ഇൻഫറൻസ് സമയത്ത് ഇൻഡക്സ് ചെയ്ത ഡോക്യുമെന്റിൽ നിന്ന് അനുയോജ്യമായ ഭാഗങ്ങൾ തിരയാനും തിരികെ ലഭിക്കാനും സഹായിക്കുന്നു.

5. **ഉപയോക്തൃ ഇടപെടൽ**: ഉപയോക്താക്കൾ ചോദ്യങ്ങൾ ചോദിക്കാം. ഡോക്യുമെന്റിൽ അനുയോജ്യമായ വിവരങ്ങൾ ലഭ്യമായാൽ, ഏജന്റ് ഒരു അടിസ്ഥാനമുള്ള ഉത്തരം സൃഷ്ടിക്കുന്നു.  
   ലഭ്യമല്ലെങ്കിൽ, ഡോക്യുമെന്റിൽ മതിയായ വിവരങ്ങൾ ഇല്ലെന്ന് ഏജന്റ് വ്യക്തമായി പ്രതികരിക്കുന്നു.


### പ്രധാന ഫംഗ്ഷൻ


ആദ്യം Azure CLI ഉപയോഗിച്ച് `az login` പ്രവർത്തിപ്പിക്കുക, അതിനാൽ `DefaultAzureCredential` ഉപയോഗിക്കുമ്പോൾ ശരിയായ authentication context ലഭ്യമാകും. Azure AI Agent Service API കീകൾ ഉപയോഗിക്കുന്നില്ല.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

നിങ്ങൾക്ക് താഴെ കാണുന്ന പോലെ ഒരു ഔട്ട്പുട്ട് കാണാൻ കഴിയും:

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖ പ്രാമാണികമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മാനവ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
